In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
import matplotlib.ticker as mtick

In [ ]:
indicators = pd.read_parquet("../../data/indicators/US/all_indicators_raw_outer.parquet", engine="pyarrow")
indicators["date"] = pd.to_datetime(indicators["date"])
indicators.reset_index(drop=True, inplace=True)

In [ ]:
nber_recessions = pd.read_parquet("../../data/indicators/US/nber_recession.parquet")
nber_recessions["date"] = pd.to_datetime(nber_recessions["date"])
nber_recessions = nber_recessions[nber_recessions["date"] >= "1962-01-01"]

In [ ]:
us_top_500 = pd.read_parquet("../../data/indicators/US/us_top_500.parquet", engine="pyarrow")
us_top_500["date"] = pd.to_datetime(us_top_500["date"])
data = pd.merge(indicators, us_top_500, on=["date"], how="outer")

In [ ]:
data.set_index("date", inplace=True)

In [ ]:
for column in data.columns:
    print(column)
    print(data[column].dropna().index.min())
    print(data[column].dropna().index.max())

In [ ]:
np.random.seed(49)

In [ ]:
data["ism_prod"] = data["ISM_prod_index"].copy()
data["vix"] = data["vix_SP500_close"].copy()
data["inflation"] = data["inflation"]/100
data.loc[data.index < pd.Timestamp("1997-01-01"), "dvps_12m"] = np.nan

In [ ]:
#Shifting appropriate date periods, daily data:
data["vix"] = data["vix"].shift(1, freq = "D")
data["market_cap_usd"] = data["market_cap_usd"].shift(1, freq = "D")
data["credit_spread"] = data["credit_spread"].shift(1, freq = "D")
data["rate_fed_funds"] = data["rate_fed_funds"].shift(1, freq = "D")
data["rate_1_year"] = data["rate_1_year"].shift(1, freq = "D")
data["rate_3_year"] = data["rate_3_year"].shift(1, freq = "D")
data["rate_5_year"] = data["rate_5_year"].shift(1, freq = "D")
data["rate_10_year"] = data["rate_10_year"].shift(1, freq = "D")

In [ ]:
#Shifting appropriate date periods, weekly data:
data["initial_claims"] = data["initial_claims"].dropna().shift(1, freq = "W")

In [ ]:
#Shifting appropriate date periods, monthly and quarterly data:
data["real_gnp"] = data["real_gnp"].dropna().shift(3 + 2, freq = "MS")
data["real_gdp"] = data["real_gdp"].dropna().shift(3 + 2, freq = "MS")
data["M1"] = data["M1"].dropna().shift(1, freq = "MS")
data["M2"] = data["M2"].dropna().shift(1, freq = "MS")
data["ism_prod"] = data["ism_prod"].resample("ME").mean().shift(1, freq="D")
data["pce"] = data["pce"].dropna().shift(1, freq = "MS").shift(7, freq = "D")
data["unemployment"] = data["unemployment"].dropna().shift(2, freq = "MS")
data["earnings_yield"] = data["earnings_yield_12m"].dropna().shift(-1, freq = "D").resample("QE").last().shift(1, freq = "D").shift(2, freq="MS")
data["dividend_yield"] = data["dividend_yield_12m"].dropna().shift(0, freq = "MS")
data["eps"] = data["eps_12m"].dropna().shift(-1, freq = "D").resample("QE").last().shift(1, freq = "D").shift(2, freq="MS")
data["dvps"] = data["dvps_12m"].dropna().shift(0, freq = "MS")
data["inflation"] = data["inflation"].dropna().shift(2, freq = "MS")

#### Data is resampled to month-end and added one day to, so the first day of the month is information from last month

In [ ]:
#Daily data, resample to monthly, pct_change
data["vix_change"] = data["vix"].resample("ME").mean().shift(1, freq="D").dropna().pct_change()
data["mc_change"] = data["market_cap_usd"].resample("ME").mean().shift(1, freq="D").dropna().pct_change()
data["credit_spread_change"] = data["credit_spread"].resample("ME").mean().shift(1, freq="D").dropna().pct_change()
data["rate_fed_funds_change"] = data["rate_fed_funds"].resample("ME").mean().shift(1, freq="D").dropna().pct_change()
data["rate_1_year_change"] = data["rate_1_year"].resample("ME").mean().shift(1, freq="D").dropna().pct_change()
data["rate_3_year_change"] = data["rate_3_year"].resample("ME").mean().shift(1, freq="D").dropna().pct_change()
data["rate_5_year_change"] = data["rate_5_year"].resample("ME").mean().shift(1, freq="D").dropna().pct_change()
data["rate_10_year_change"] = data["rate_10_year"].resample("ME").mean().shift(1, freq="D").dropna().pct_change()

In [ ]:
#Weekly data, resample to monthly, pct_change
data["initial_claims_change"] = data["initial_claims"].resample("ME").mean().shift(1, freq="D").dropna().pct_change()

In [ ]:
#Monthly data, pct_change

data["real_gnp_change"] = data["real_gnp"].dropna().pct_change()
data["real_gdp_change"] = data["real_gdp"].dropna().pct_change()
data["m1_change"] = data["M1"].dropna().pct_change()
data["m2_change"] = data["M2"].dropna().pct_change()
data["ism_prod_change"] = data["ism_prod"].dropna().pct_change()
data["pce_change"] = data["pce"].dropna().pct_change()
data["unemployment_change"] = data["unemployment"].dropna().pct_change()
data["earnings_yield_change"] = data["earnings_yield"].dropna().pct_change()
data["dividend_yield_change"] = data["dividend_yield"].dropna().pct_change()
data["eps_change"] = data["eps"].dropna().pct_change()
data["dvps_change"] = data["dvps"].dropna().pct_change()
data["inflation_change"] = data["inflation"].dropna().pct_change()

In [ ]:
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import StandardScaler
def save_data(data, feature, train_suffix, test_suffix = None, first_test_date = None, winsorize_std = 3, winsorize_quantile=None, scale_data=False, log_transform=False):
    assert(train_suffix is not None)
    assert(not (winsorize_std and winsorize_quantile))
    
    data_train = data[[feature]].copy().dropna()
    print(data_train.head())

    if winsorize_quantile is not None:
        data_train[feature] = data_train[feature].clip(lower = data_train[feature].quantile(winsorize_quantile), upper = data_train[feature].quantile(1-winsorize_quantile))

    if winsorize_std is not None:
        train_std = data_train[feature].std()
        data_train[feature] = data_train[feature].clip(lower = -train_std*winsorize_std, upper = train_std*winsorize_std)
        data_test[feature] = data_test[feature].clip(lower = -train_std*winsorize_std, upper = train_std*winsorize_std)

    scaler = StandardScaler()

    if first_test_date is not None:
        data_test = data_train[data_train.index >= first_test_date]
        data_train = data_train[data_train.index < first_test_date]

    if data_train.shape[0] == 0:
        print("No data for feature after test date, saving empty dataframe")
        pd.DataFrame(data_train).to_csv(f"../../data/indicators/US/matlab_ready/{feature}_train_{train_suffix}.csv")
        if first_test_date is not None:
            pd.DataFrame(data_test).to_csv(f"../../data/indicators/US/matlab_ready/{feature}_test_{test_suffix}.csv")
            data_all = pd.concat([data_train, data_test])
            pd.DataFrame(data_all).to_csv(f"../../data/indicators/US/matlab_ready/{feature}_all_{train_suffix}.csv")

        return

    if log_transform:
        if (data_train[feature].min() + 1) <= 0:
            print(data_train[feature].min())
            print(f"Feature {feature} has too negative values, cannot log transform")
            return
        data_train[feature] = np.log(1 + data_train[feature])
        if first_test_date is not None:
            data_test[feature] = np.log(1 + data_test[feature])

    if scale_data:
        data_train[feature] = scaler.fit_transform(data_train[feature].values.reshape(-1, 1))
        if first_test_date is not None:
            data_test[feature] = scaler.transform(data_test[feature].values.reshape(-1, 1))

    print(feature)
    if first_test_date is not None:
        data_all = pd.concat([data_train, data_test])
        data_all[feature].plot()
    else:
        data_train[feature].plot()

    plt.show()

    pd.DataFrame(data_train).to_csv(f"../../data/indicators/US/matlab_ready/{feature}_train_{train_suffix}.csv")
    if first_test_date is not None:
        pd.DataFrame(data_test).to_csv(f"../../data/indicators/US/matlab_ready/{feature}_test_{test_suffix}.csv")
        
        pd.DataFrame(data_all).to_csv(f"../../data/indicators/US/matlab_ready/{feature}_all_{train_suffix}.csv")

### Data is processed and saved to be used in the matlab script to fit the models

In [ ]:
#Only change variables:
features_to_save = ["vix_change", "mc_change", "credit_spread_change",
                    "rate_fed_funds_change",
                    "rate_1_year_change", "rate_3_year_change", "rate_5_year_change", "rate_10_year_change", 
                    "initial_claims_change", 
                    "real_gnp_change", "real_gdp_change", "m1_change", "m2_change", 
                    "ism_prod_change", "pce_change", "unemployment_change",
                    "earnings_yield_change", "dividend_yield_change", 
                    #"inflation_change"
                    "inflation",
                    "dvps_change", "eps_change"
                    ]

In [ ]:
data_copy = data.copy()
data_copy.index = pd.to_datetime(data_copy.index)

year_to_save = 2020

train_suffix = f"scale_win3std_log_{year_to_save}"
test_suffix = f"scale_win3std_log_{year_to_save}"

first_test_date = pd.Timestamp(f"{year_to_save}-01-01")

for feature in features_to_save:
    save_data(data_copy, feature, train_suffix, test_suffix, first_test_date, winsorize_std=3, winsorize_quantile=None, scale_data=True, log_transform=True)

In [ ]:
endog_variables = [
                "mc_change", 
                "inflation",
                #"inflation_change",
                #"unemployment", 
                "unemployment_change", 
                #"rate_fed_funds",
                "rate_fed_funds_change", 
                "initial_claims_change",
                #"ism_prod_index",
                "ism_prod_change",
                #"real_gnp_change", 
                #"real_gdp_change", 
                "m1_change", 
                "m2_change", 
                #"rate_1_year",
                #"rate_3_year",
                #"rate_5_year",
                #"rate_10_year",
                "rate_1_year_change",
                #"rate_3_year_change",
                #"rate_5_year_change",
                "rate_10_year_change",
                #"earnings_yield",
                "eps_change",
                #"dvps_change",
                "earnings_yield_change",
                #"dividend_yield_change",
                #"credit_spread",
                "credit_spread_change",
                #"pce_change",
                "vix_change"
                    ]

In [ ]:
year = 2020

prefix = "train"

suffix = "_smooth"

#suffix = "_train_1990"
#suffix = "_all_win_scaled_order4"
suffix = f"_test_scale_win3std_log_{year}_order4_smooth"

suffixes = [f"_{prefix}_scale_win3std_log_{year}_order1{suffix}", f"_{prefix}_scale_win3std_log_{year}_order4{suffix}", f"_{prefix}_scale_win3std_log_{year}_order10{suffix}"]

In [ ]:
train_test_split_date = pd.Timestamp(f"{year}-01-01")

### Results are here imported from the directory where MATLAB saved them

In [ ]:
#Results from matlab:
all_probabilities = pd.DataFrame(columns=["endog", "order", "date", "p"])

for current_endog in endog_variables:
    print(current_endog)
    for suffix in suffixes:
        current_results = pd.read_csv("../../results/regime/markov_matlab/" + current_endog + suffix + ".csv")
        current_results["date"] = pd.to_datetime(current_results["date"], format="mixed")
        current_results["order"] = int(suffix.split("order")[1])
        current_results["endog"] = current_endog

        all_probabilities = pd.concat([all_probabilities, current_results], axis=0)

all_probabilities["date"] = pd.to_datetime(all_probabilities["date"])

### Collect all probabilities from the test results into one DataFrame

In [ ]:
split_years = list(range(1980,2020+1,5))
orders = [1,4,10]
prefix = "all"
flip_probs = False


for year_i, year in enumerate(split_years):
    for endog_i, current_endog in enumerate(endog_variables):
        print(year, current_endog)
        found_file = False
        for order_i, order in enumerate(orders):
            if not os.path.exists("../../results/regime/markov_matlab/" + current_endog + f"_{prefix}_scale_win3std_log_{year}_order{order}" + ".csv"):
                print(year, current_endog, order, " does not exist")
                continue
            found_file = True

            current_results = pd.read_csv("../../results/regime/markov_matlab/" + current_endog + f"_{prefix}_scale_win3std_log_{year}_order{order}" + ".csv")
            current_results["date"] = pd.to_datetime(current_results["date"], format="mixed")
            current_results["endog"] = current_endog
            if order_i == 0:
                avg_results = current_results
                continue
            current_results_train = current_results[current_results["date"] < pd.Timestamp(f"{year}-01-01")]
            avg_results_train = avg_results[avg_results["date"] < train_test_split_date]
            if flip_probs:
                if ((current_results_train["p"].mean() > 0.5) and (avg_results_train["p"].mean() < 0.5)) or ((current_results_train["p"].mean() < 0.5) and (avg_results_train["p"].mean() > 0.5)):
                    print("Flipping", current_endog, suffix)
                    current_results["p"] = 1 - current_results["p"]
                    
            avg_results = pd.concat([avg_results, current_results])
        if not found_file:
            continue
            
        avg_results = avg_results.groupby(["endog", "date"]).mean().reset_index()
        avg_results["split_date"] = pd.Timestamp(f"{year}-01-01")
        if year != split_years[0]:
            avg_results = avg_results[avg_results["date"] >= pd.Timestamp(f"{year}-01-01")]
        if year != split_years[-1]:
            avg_results = avg_results[avg_results["date"] < (pd.Timestamp(f"{year}-01-01") + pd.DateOffset(years=5))]
                
        if year_i == 0 and endog_i == 0:
            test_probabilites = avg_results
        else:
            test_probabilites = pd.concat([test_probabilites, avg_results])
            

test_probabilites.sort_values(["date", "endog"], inplace=True)  

In [ ]:
test_probabilites.to_csv("../../results/regime/markov_collected/probabilities_scale_win3std_log_test_all_years_order1_4_10.csv")

### Get smoothed in-sample probabilites

In [ ]:
#Results from matlab:

flip_probs = False

probabilities = pd.DataFrame(columns=["endog", "date", "p"])

for current_endog in endog_variables:
    print(current_endog)
    avg_results = pd.DataFrame(columns=["endog", "date", "p"])
    for suffix in suffixes:
        current_results = pd.read_csv("../../results/regime/markov_matlab/" + current_endog + suffix + ".csv")
        current_results["date"] = pd.to_datetime(current_results["date"], format="mixed")
        current_results_train = current_results[current_results["date"] < train_test_split_date]
        avg_results_train = avg_results[avg_results["date"] < train_test_split_date]
        if flip_probs:
            if ((current_results_train["p"].mean() > 0.5) and (avg_results_train["p"].mean() < 0.5)) or ((current_results_train["p"].mean() < 0.5) and (avg_results_train["p"].mean() > 0.5)):
                print("Flipping", current_endog, suffix)
                current_results["p"] = 1 - current_results["p"]
        current_results["endog"] = current_endog
        current_results["date"] = pd.to_datetime(current_results["date"])
        avg_results = pd.concat([avg_results, current_results], axis=0)
    avg_results = avg_results.groupby(["endog", "date"]).mean().reset_index()
    
    probabilities = pd.concat([probabilities, avg_results], axis=0)

probabilities["date"] = pd.to_datetime(probabilities["date"])

In [ ]:
endog_variables_display = endog_variables

### These are imported after they are collected and saved

In [ ]:
probabilities = pd.read_csv("../../results/regime/markov_collected/probabilities_scale_win3std_log_train_2020_order1_4_10_smooth.csv")
probabilities["date"] = pd.to_datetime(probabilities["date"])

In [ ]:
probabilities = pd.read_csv("../../results/regime/markov_collected/probabilities_scale_win3std_log_test_all_years_order1_4_10.csv")
probabilities["date"] = pd.to_datetime(probabilities["date"])

In [ ]:
for endog in endog_variables_display:
    print(endog)
    print(probabilities[probabilities["endog"] == endog].dropna()["date"].min())

In [ ]:
markov_rec_dates = pd.read_csv("../../time_periods/model_train_ready_before_test/markov_rec_dates_train_2020_order1_4_10_smooth_5yr_avg.csv")
markov_rec_dates["date"] = pd.to_datetime(markov_rec_dates["date"])

In [ ]:
markov_exp_dates = pd.read_csv("../../time_periods/model_train_ready_before_test/markov_exp_dates_train_2020_order1_4_10_smooth_5yr_avg.csv")
markov_exp_dates["date"] = pd.to_datetime(markov_exp_dates["date"])

In [ ]:
nber_rec_dates = pd.read_csv("../../time_periods/model_train_ready/nber_recession_dates.csv")
nber_rec_dates["date"] = pd.to_datetime(nber_rec_dates["date"])

In [ ]:
fig, axes = plt.subplots(2, figsize=(15, 8), sharex=False)

ax = axes[0]

expansion = False

probabilities = probabilities.copy()



data_copy = data.copy()

min_date = pd.Timestamp("1975-01-01")

flip_probs = False

resample_freq = "MS"


probabilities_display = probabilities[probabilities["date"] > min_date]

if expansion:
    probabilities_display["p"] = 1-probabilities_display["p"]
    

if flip_probs:
    for endog in endog_variables_display:
        if probabilities_display[probabilities_display["endog"] == endog]["p"].mean() > 0.5:
            print(endog, " is flipped")
            probabilities_display[probabilities_display["endog"] == endog]["p"] = 1 - probabilities_display[probabilities_display["endog"] == endog]["p"]

market_cap = data_copy[data_copy.index > probabilities_display["date"].min()]["market_cap_usd"].resample(resample_freq).first()


for current_endog in endog_variables_display:
    print(current_endog)
    print(probabilities_display[probabilities_display["endog"] == current_endog].shape)
    probabilities_display[probabilities_display["endog"] == current_endog].plot(x="date", y="p", ax=ax)

ax.axvline(pd.Timestamp("1980-01-01"), color="black", linestyle="--", label="Fi", linewidth=2)

current_i = 0
for i in range(len(nber_rec_dates['date'])-1):
    if nber_rec_dates['date'].iloc[i+1] - pd.DateOffset(days=1) == nber_rec_dates['date'].iloc[i]:
        continue
    ax.axvspan(nber_rec_dates['date'].iloc[current_i], nber_rec_dates['date'].iloc[i] + pd.DateOffset(days=1), facecolor='grey', alpha=0.5)
    current_i = i + 1


ax.get_legend().remove()

ax = axes[1]

ax2 = ax.twinx()
market_cap.plot(ax=ax2, alpha=0.5, color="tab:orange", label="Index Market Cap (Log)", logy=True, linewidth=2)

avg_probabilities = probabilities_display[probabilities_display["endog"].isin(endog_variables_display)].groupby("date")["p"].mean()
avg_probabilities.plot(ax=ax, color="tab:blue", label="average", linewidth=2)

axvlines = [pd.Timestamp("1980-01-01"), pd.Timestamp("1985-01-01"), pd.Timestamp("1990-01-01"), 
                        pd.Timestamp("1995-01-01"), pd.Timestamp("2000-01-01"), pd.Timestamp("2005-01-01"),
                        pd.Timestamp("2010-01-01"), pd.Timestamp("2015-01-01"), pd.Timestamp("2020-01-01")]

for line in axvlines:
    ax.axvline(line, color="black", linestyle="--", linewidth=2)

current_i = 0
for i in range(len(nber_rec_dates['date'])-1):
    if nber_rec_dates['date'].iloc[i+1] - pd.DateOffset(days=1) == nber_rec_dates['date'].iloc[i]:
        continue
    ax.axvspan(nber_rec_dates['date'].iloc[current_i], nber_rec_dates['date'].iloc[i] + pd.DateOffset(days=1), facecolor='grey', alpha=0.5)
    current_i = i + 1
ax.axvspan(nber_rec_dates['date'].iloc[current_i], nber_rec_dates['date'].iloc[i] + pd.DateOffset(days=1), facecolor='grey', alpha=0.5)

current_i = 0
for i in range(len(markov_rec_dates['date'])-1):
    if markov_rec_dates['date'].iloc[i+1] - pd.DateOffset(days=1) == markov_rec_dates['date'].iloc[i]:
        continue
    ax.axvspan(markov_rec_dates['date'].iloc[current_i], markov_rec_dates['date'].iloc[i] + pd.DateOffset(days=1), facecolor='red', alpha=0.3, ymin=0.5, ymax=0.9)
    current_i = i + 1
ax.axvspan(markov_rec_dates['date'].iloc[current_i], markov_rec_dates['date'].iloc[i] + pd.DateOffset(days=1), facecolor='red', alpha=0.3, ymin=0.5, ymax=0.9)

    
current_i = 0
for i in range(len(markov_exp_dates['date'])-1):
    if markov_exp_dates['date'].iloc[i+1] - pd.DateOffset(days=1) == markov_exp_dates['date'].iloc[i]:
        continue
    ax.axvspan(markov_exp_dates['date'].iloc[current_i], markov_exp_dates['date'].iloc[i] + pd.DateOffset(days=1), facecolor='blue', alpha=0.3, ymin=0.1, ymax=0.5)
    current_i = i + 1
ax.axvspan(markov_exp_dates['date'].iloc[current_i], markov_exp_dates['date'].iloc[i] + pd.DateOffset(days=1), facecolor='blue', alpha=0.3, ymin=0.1, ymax=0.5)


axes[0].tick_params(axis='both', which='major', labelsize=14)
axes[1].tick_params(axis='both', which='major', labelsize=14)

ax2.axes.get_yaxis().set_ticks([])


axes[0].axes.get_xaxis().set_label_text('')
axes[1].axes.get_xaxis().set_label_text('')
axes[0].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
axes[1].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

plt.tight_layout()

In [ ]:
fig.savefig("../../figures/markov_regression_smooth_2020.pdf", dpi=3000)

In [ ]:
markov_rec_filter_dates = pd.read_csv("../../time_periods/model_test_ready/markov_rec_dates_test_all_years_order1_4_10_5yr_avg.csv")
markov_rec_filter_dates["date"] = pd.to_datetime(markov_rec_filter_dates["date"])

In [ ]:
markov_exp_filter_dates = pd.read_csv("../../time_periods/model_test_ready/markov_exp_dates_test_all_years_order1_4_10_5yr_avg.csv")
markov_exp_filter_dates["date"] = pd.to_datetime(markov_exp_filter_dates["date"])

In [ ]:
test_probabilites = pd.read_csv("../../results/regime/markov_collected/probabilities_scale_win3std_log_test_all_years_order1_4_10.csv")
test_probabilites["date"] = pd.to_datetime(test_probabilites["date"])

In [ ]:
#Only result plot

fig, ax = plt.subplots(1, figsize=(12, 4), sharex=False)


expansion = False

#probabilities = test_probabilites.copy()

data_copy = data.copy()

min_date = pd.Timestamp("1975-01-01")

flip_probs = False

resample_freq = "MS"


probabilities_display = probabilities[probabilities["date"] > min_date]

if expansion:
    probabilities_display["p"] = 1-probabilities_display["p"]
    

if flip_probs:
    for endog in endog_variables_display:
        print(endog)
        if probabilities_display[probabilities_display["endog"] == endog]["p"].mean() > 0.5:
            print(endog, " is flipped")
            probabilities_display[probabilities_display["endog"] == endog]["p"] = 1 - probabilities_display[probabilities_display["endog"] == endog]["p"]

market_cap = data_copy[data_copy.index > probabilities_display["date"].min()]["market_cap_usd"].resample(resample_freq).first()



avg_probabilities = probabilities_display[probabilities_display["endog"].isin(endog_variables_display)].groupby("date")["p"].mean()
avg_probabilities.plot(ax=ax, color="tab:blue", label="average", linewidth=2)

axvlines = [pd.Timestamp("1980-01-01"), pd.Timestamp("1985-01-01"), pd.Timestamp("1990-01-01"), 
                        pd.Timestamp("1995-01-01"), pd.Timestamp("2000-01-01"), pd.Timestamp("2005-01-01"),
                        pd.Timestamp("2010-01-01"), pd.Timestamp("2015-01-01"), pd.Timestamp("2020-01-01")]

for line in axvlines:
    ax.axvline(line, color="black", linestyle="--", linewidth=2)

current_i = 0
for i in range(len(nber_rec_dates['date'])-1):
    if nber_rec_dates['date'].iloc[i+1] - pd.DateOffset(days=1) == nber_rec_dates['date'].iloc[i]:
        continue
    ax.axvspan(nber_rec_dates['date'].iloc[current_i], nber_rec_dates['date'].iloc[i] + pd.DateOffset(days=1), facecolor='grey', alpha=0.5)
    current_i = i + 1
ax.axvspan(nber_rec_dates['date'].iloc[current_i], nber_rec_dates['date'].iloc[i] + pd.DateOffset(days=1), facecolor='grey', alpha=0.5)


current_negative_dates = markov_rec_dates

current_i = 0
for i in range(len(current_negative_dates['date'])-1):
    if current_negative_dates['date'].iloc[i+1] - pd.DateOffset(days=1) == current_negative_dates['date'].iloc[i]:
        continue
    ax.axvspan(current_negative_dates['date'].iloc[current_i], current_negative_dates['date'].iloc[i] + pd.DateOffset(days=1), facecolor='red', alpha=0.3, ymin=0.5, ymax=0.9)
    current_i = i + 1
ax.axvspan(current_negative_dates['date'].iloc[current_i], current_negative_dates['date'].iloc[i] + pd.DateOffset(days=1), facecolor='red', alpha=0.3, ymin=0.5, ymax=0.9)

current_positive_dates = markov_exp_dates

current_i = 0
for i in range(len(current_positive_dates['date'])-1):
    if current_positive_dates['date'].iloc[i+1] - pd.DateOffset(days=1) == current_positive_dates['date'].iloc[i]:
        continue
    ax.axvspan(current_positive_dates['date'].iloc[current_i], current_positive_dates['date'].iloc[i] + pd.DateOffset(days=1), facecolor='blue', alpha=0.3, ymin=0.1, ymax=0.5)
    current_i = i + 1
ax.axvspan(current_positive_dates['date'].iloc[current_i], current_positive_dates['date'].iloc[i] + pd.DateOffset(days=1), facecolor='blue', alpha=0.3, ymin=0.1, ymax=0.5)

ax2 = ax.twinx()
market_cap.plot(ax=ax2, alpha=0.5, color="tab:orange", label="Index Market Cap (Log)", logy=True, linewidth=2)


ax.tick_params(axis='both', which='major', labelsize=14)

ax2.axes.get_yaxis().set_ticks([])
ax.set_ylim(0, 1)

ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.axes.get_xaxis().set_label_text('')

plt.tight_layout()

In [ ]:

fig.savefig("../../figures/markov_regression_oos.pdf")

In [ ]:
#Plot every endog variable separately
for name, group in probabilities.groupby('endog'):
    plt.figure()
    group.plot(x='date', y='p', title=f"Probabilities for Endog: {name}")
    plt.show()

In [ ]:
collected_files = os.listdir("../../results/regime/markov_collected/")
save_years = list(range(2020,2020+1, 5))

In [ ]:
for year in save_years:
    file = [file for file in collected_files if str(year) in file][0]
    if "probabilities" not in file:
        continue
    print(file)
    current_probabilites = pd.read_csv("../../results/regime/markov_collected/" + file)
    current_probabilites["date"] = pd.to_datetime(current_probabilites["date"])
    if (current_probabilites["date"].dt.day.unique().item() != 1):
        print("Not centered on MS")
        break
    #Recession
    avg_probabilities = current_probabilites.groupby("date")["p"].mean()
    avg_probabilities = pd.DataFrame(avg_probabilities)
    avg_probabilities["class"] = 0
    avg_probabilities.loc[avg_probabilities[avg_probabilities["p"] > avg_probabilities["p"].rolling(f'{5*12*30}D').mean()].index, "class"] = 1
    avg_probabilities_daily = avg_probabilities.resample("D").ffill()
    avg_probabilities_daily.loc[pd.Timestamp(f"{year-1}-12-31")] = avg_probabilities_daily.loc[pd.Timestamp(f"{year-1}-12-01")]
    avg_probabilities_daily = avg_probabilities_daily.resample("D").ffill()
    
    pd.DataFrame(avg_probabilities_daily[avg_probabilities_daily["class"] == 1].index, columns=["date"]).to_csv(f"../../time_periods/model_train_ready_before_test/markov_rec_dates_train_{year}_order1_4_10_smooth_5yr_avg.csv", index=False)
    
    #Expansion
    expansion_probabilities = current_probabilites.copy()
    expansion_probabilities["p"] = 1 - expansion_probabilities["p"]
    avg_probabilities = expansion_probabilities.groupby("date")["p"].mean()
    avg_probabilities = pd.DataFrame(avg_probabilities)
    avg_probabilities["class"] = 0
    avg_probabilities.loc[avg_probabilities[avg_probabilities["p"] > avg_probabilities["p"].rolling(f'{5*12*30}D').mean()].index, "class"] = 1
    avg_probabilities_daily = avg_probabilities.resample("D").ffill()
    avg_probabilities_daily.loc[pd.Timestamp(f"{year-1}-12-31")] = avg_probabilities_daily.loc[pd.Timestamp(f"{year-1}-12-01")]
    avg_probabilities_daily = avg_probabilities_daily.resample("D").ffill()
    pd.DataFrame(avg_probabilities_daily[avg_probabilities_daily["class"] == 1].index, columns=["date"]).to_csv(f"../../time_periods/model_train_ready_before_test/markov_exp_dates_train_{year}_order1_4_10_smooth_5yr_avg.csv", index=False)

In [ ]:
current_probabilites = pd.read_csv("../../results/regime/markov_collected/probabilities_scale_win3std_log_test_all_years_order1_4_10.csv")

current_probabilites["date"] = pd.to_datetime(current_probabilites["date"])
if (current_probabilites["date"].dt.day.unique().item() != 1):
    print("Not centered on MS")
else:
    #Recession
    avg_probabilities = current_probabilites.groupby("date")["p"].mean()
    avg_probabilities = pd.DataFrame(avg_probabilities)
    avg_probabilities["class"] = 0
    avg_probabilities.loc[avg_probabilities[avg_probabilities["p"] > avg_probabilities["p"].rolling(f'{5*12*30}D').mean()].index, "class"] = 1
    avg_probabilities_daily = avg_probabilities.resample("D").ffill()
    avg_probabilities_daily.loc[pd.Timestamp(f"{year-1}-12-31")] = avg_probabilities_daily.loc[pd.Timestamp(f"{year-1}-12-01")]
    avg_probabilities_daily = avg_probabilities_daily.resample("D").ffill()

    pd.DataFrame(avg_probabilities_daily[avg_probabilities_daily["class"] == 1].index, columns=["date"]).to_csv(f"../../results/regime/markov_test/markov_rec_dates_test_all_years_order1_4_10_5yr_avg.csv", index=False)

    #Expansion
    expansion_probabilities = current_probabilites.copy()
    expansion_probabilities["p"] = 1 - expansion_probabilities["p"]
    avg_probabilities = expansion_probabilities.groupby("date")["p"].mean()
    avg_probabilities = pd.DataFrame(avg_probabilities)
    avg_probabilities["class"] = 0
    avg_probabilities.loc[avg_probabilities[avg_probabilities["p"] > avg_probabilities["p"].rolling(f'{5*12*30}D').mean()].index, "class"] = 1
    avg_probabilities_daily = avg_probabilities.resample("D").ffill()
    avg_probabilities_daily.loc[pd.Timestamp(f"{year-1}-12-31")] = avg_probabilities_daily.loc[pd.Timestamp(f"{year-1}-12-01")]
    avg_probabilities_daily = avg_probabilities_daily.resample("D").ffill()
    pd.DataFrame(avg_probabilities_daily[avg_probabilities_daily["class"] == 1].index, columns=["date"]).to_csv(f"../../results/regime/markov_test/markov_exp_dates_test_all_years_order1_4_10_5yr_avg.csv", index=False)